# Daily Stop-Loss: Risk Management with ProbFlow

In this tutorial, we use ProbFlow to build a risk management model.
We model daily portfolio returns with a **Normal** distribution,
compute **Value-at-Risk** (VaR) using the quantile function, and
use **joint distributions** to analyze correlated portfolio positions.

## Setup

In [ ]:
import numpy as np
from probflow import Normal, LogNormal, ProbFlow

## Modeling Daily Returns

Daily portfolio returns are commonly modeled as normally distributed.
We set `loc=0.0005` (5 basis points mean daily return) and
`scale=0.02` (2% daily volatility).

In [ ]:
daily_returns = Normal(loc=0.0005, scale=0.02)
returns = daily_returns.sample(252)
print(f"Simulated 252 trading days")
print(f"Mean daily return: {np.mean(returns):.6f}")
print(f"Std dev: {np.std(returns):.6f}")

## Value-at-Risk (VaR)

VaR answers: "What is the maximum loss we expect on 95% (or 99%) of days?"
It is the quantile of the return distribution at the desired confidence level.
A negative VaR value indicates a loss.

In [ ]:
var_95 = daily_returns.quantile(0.05)
var_99 = daily_returns.quantile(0.01)
print(f"95% VaR: {var_95:.4f} ({var_95*100:.2f}%)")
print(f"99% VaR: {var_99:.4f} ({var_99*100:.2f}%)")

## Stop-Loss Trigger Probability

A daily stop-loss is triggered when the loss exceeds a threshold.
If our stop-loss is set at -3%, what is the probability of triggering it?

In [ ]:
stop_loss_level = -0.03
prob_trigger = daily_returns.cdf(stop_loss_level)
print(f"P(return < {stop_loss_level*100}%) = {prob_trigger:.6f}")
print(f"Expected trigger frequency: ~1 in {1/prob_trigger:.0f} days")

## PDF of Returns

We evaluate the probability density at specific return levels.

In [ ]:
test_returns = np.array([-0.04, -0.02, 0.0, 0.02, 0.04])
densities = daily_returns.pdf(test_returns)
for r, d in zip(test_returns, densities):
    print(f"  PDF at {r*100:+.0f}%: {d:.4f}")

## Two-Asset Portfolio

We model a portfolio with two independent positions. Asset A is a
low-volatility equity, and Asset B is a higher-volatility commodity.

In [ ]:
asset_a = Normal(loc=0.0003, scale=0.015)
asset_b = Normal(loc=0.0008, scale=0.035)
print("Asset A: Normal(mean=0.03%, vol=1.5%)")
print("Asset B: Normal(mean=0.08%, vol=3.5%)")

## Joint Distribution of Returns

Using the `&` operator, we create a joint distribution of both assets.
Each sample is a pair `[return_A, return_B]`.

In [ ]:
joint_returns = asset_a & asset_b
joint_samples = joint_returns.sample(5000)
print(f"Joint samples shape: {joint_samples.shape}")
print(f"Asset A mean: {np.mean(joint_samples[:, 0]):.6f}")
print(f"Asset B mean: {np.mean(joint_samples[:, 1]):.6f}")

## Portfolio Return (Equal Weight)

With equal weights, the portfolio return is the sum of the individual
returns (scaled by 0.5 each). We use the `+` operator to combine them
directly, which gives us the sum distribution.

In [ ]:
portfolio = asset_a + asset_b
portfolio_samples = portfolio.sample(10000)
print(f"Portfolio mean return: {np.mean(portfolio_samples):.6f}")
print(f"Portfolio std dev: {np.std(portfolio_samples):.6f}")

## Portfolio VaR

We compute VaR for the combined portfolio using the quantile function
on the sum distribution.

In [ ]:
portfolio_var_95 = portfolio.quantile(0.05)
portfolio_var_99 = portfolio.quantile(0.01)
print(f"Portfolio 95% VaR: {portfolio_var_95:.4f}")
print(f"Portfolio 99% VaR: {portfolio_var_99:.4f}")

## Joint PDF Evaluation

We evaluate the joint PDF at specific return pairs. Under independence,
the joint PDF is the product of marginal PDFs.

In [ ]:
test_pairs = np.array([[0.0, 0.0], [0.01, 0.01], [-0.02, 0.02]])
joint_pdf = joint_returns.pdf(test_pairs)
for pair, p in zip(test_pairs, joint_pdf):
    print(f"  PDF at ({pair[0]*100:+.0f}%, {pair[1]*100:+.0f}%): {p:.4f}")

## Risk Management Model

We define the complete risk model using the ProbFlow context manager.

In [ ]:
with ProbFlow() as risk_model:
    equity = Normal(loc=0.0003, scale=0.015)
    risk_model.add_distribution(equity, name="equity_returns")

    commodity = Normal(loc=0.0008, scale=0.035)
    risk_model.add_distribution(commodity, name="commodity_returns")

print(f"Risk model components: {list(risk_model.variables.keys())}")

## Computing Stop-Loss from the Model

We retrieve each position and compute individual stop-loss levels.

In [ ]:
for name in ["equity_returns", "commodity_returns"]:
    dist = risk_model.get_distribution(name)
    var95 = dist.quantile(0.05)
    var99 = dist.quantile(0.01)
    print(f"{name}: 95% VaR={var95:.4f}, 99% VaR={var99:.4f}")

## Summary

In this tutorial we:
- Modeled daily returns with a **Normal** distribution
- Computed **Value-at-Risk** using the **quantile** function
- Calculated stop-loss trigger probabilities with the **CDF**
- Created a **joint distribution** with `&` for multi-asset portfolios
- Composed returns with `+` for portfolio-level analysis
- Built a risk model with the **ProbFlow** context manager